In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from esis.data import level_3
from kgpy.img.coalignment import image_coalignment as img_align
import scipy.signal

plt.rcParams['figure.figsize'] = [10, 10]


## ESIS Level 3 Vignetting Correction
A discrepancy between our optical models prediction of the vignetting function my initial fits has led me
to run my correction routines test images with a known vignetting.  To do so I have simply replaced each ESIS image with
the best co-temporal AIA 304 image within the Level_3 object.



In [2]:
lev3 = level_3.Level_3.from_pickle(level_3.masked_pickle)
test_cube = lev3.to_test_object()

AttributeError: module 'esis.data.level_3' has no attribute 'masked_pickle'

In [ ]:
sequence = 15

fig, ax = plt.subplots(2,2,subplot_kw={'projection': test_cube.observation.wcs[sequence,0]})

z = 0
for i in range(2):
    for j in range(2):
        mimg = test_cube.observation.data[sequence,z]*test_cube.observation.mask[sequence,z]
        ax[i,j].imshow(mimg,origin = 'lower',vmax = np.percentile(mimg,99.9))
        z += 1
plt.show()

Next we will apply a vignetting field of known scale to the cube.

In [ ]:
kernel = np.zeros((11,11))
origin = (5,5)
width_x,width_y= (.5,8)
x,y = img_align.get_img_coords(kernel)
x -= origin[0]
y -= origin[1]

angles = -(np.arange(4))*45 - 22.5 -90
angles = np.deg2rad(angles)

gaussians = []
for angle in angles:
    x_prime = x*np.cos(angle) + y*np.sin(angle)
    y_prime = x*np.sin(angle) - y*np.cos(angle)
    gaussians.append(np.exp(-(np.square(x_prime)/width_x+np.square(y_prime)/width_y)))

z = 0
fig,ax = plt.subplots(2,2)

for i in range(2):
    for j in range(2):
        ax[i,j].imshow(gaussians[z])
        z+=1

for l3_seq,l1_seq in enumerate(test_cube.lev1_sequences):
    for l3_cam,l1_cam in enumerate(test_cube.lev1_cameras):
            test_cube.observation.data[l3_seq,l3_cam] = scipy.signal.convolve2d(
                test_cube.observation.data[l3_seq,l3_cam],gaussians[l3_cam]/gaussians[l3_cam].sum(),mode='same')



In [ ]:
scale = np.array([.25,.4,.3,.5])

vignetting = test_cube.correct_vignetting(scale)

test_cube.observation.data[...] *= vignetting
gains = [.75,3,1.5,.3]
noise_amp = 10
for i,gain in enumerate(gains):
    test_cube.observation.data[:,i] *= gain

test_cube.observation.data[...] += (noise_amp*np.random.random(test_cube.observation.data[...].shape)-noise_amp/2)


fig, ax = plt.subplots(2,2,subplot_kw={'projection': test_cube.observation.wcs[sequence,0]})

z = 0
for i in range(2):
    for j in range(2):
        mimg = test_cube.observation.data[sequence,z]*test_cube.observation.mask[sequence,z]
        ax[i,j].imshow(mimg,origin = 'lower',vmax = 500)
        z += 1
plt.show()

In [ ]:
normalized_vignetted_imgs = test_cube.observation.data / test_cube.masked_mean_normalization()

scale = 1

fig, ax = plt.subplots()
ax.imshow(normalized_vignetted_imgs[sequence,2]-normalized_vignetted_imgs[sequence,1], vmin = -scale,vmax = scale)
plt.show()


In [ ]:
fit = level_3.find_vignetting_correction(test_cube)
print(fit)
vignette_correction = test_cube.correct_vignetting(fit.x)

test_cube.observation.data[...] /= vignette_correction
test_cube.observation.data[...] /= test_cube.masked_mean_normalization()



In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_cube.observation.data[sequence,2]-test_cube.observation.data[sequence,1], vmin = -scale,vmax = scale)
plt.show()